# Task A — Supplier Data Cleaning

The goal of this task is to clean and normalize supplier datasets and join them into a single table `inventory_dataset.csv`.  

Both suppliers provide similar but not identical information:  

- **Supplier 1** includes Quality/Choice, Grade, Finish, Thickness, Width, Description, Gross Weight, Quantity, and mechanical properties (RP02, RM, AG, AI).  
- **Supplier 2** includes Material, Description, Article ID, Weight, Quantity, and Reserved status.  

I cleaned and standardized the datasets into a common schema, handled missing values, and ensured consistent naming.  

In [24]:
import pandas as pd
import numpy as np

### Loading the Supplier Data Files

In [25]:
s1 = pd.read_excel("../data/supplier_data1.xlsx", dtype=str)
s2 = pd.read_excel("../data/supplier_data2.xlsx", dtype=str)

In [26]:
print("Supplier 1 shape:", s1.shape)
print("Supplier 2 shape:", s2.shape)

Supplier 1 shape: (50, 12)
Supplier 2 shape: (50, 6)


### Displaying first 5 rows

In [27]:
display(s1.head())
display(s2.head())

,Quality/Choice,Grade,Finish,Thickness (mm),Width (mm),Description,Gross weight (kg),RP02,RM,Quantity,AG,AI
0,3rd,C200S,gebeizt und geglüht,2.77,1100,Längs- oder Querisse,13983,333.6,606.2,0,16.11,0.0054
1,3rd,C300S,ungebeizt,2.65,1075,Längs- oder Querisse,13047,717.7,0,0,16.11,0.0046
2,3rd,C100S,gebeizt und geglüht,2.2,1100,Kantenfehler - FS-Kantenrisse,14155,368.9,0,10.84,0,0.0061
3,2nd,C100S,gebeizt,2.86,1100,Längs- oder Querisse,11381,368.9,601.7,22.87,0,0.0062
4,1st,C300S,ungebeizt,2.88,1050,Sollmasse (Gewicht) unterschritten,10072,0,1213,22.87,0,0.0041


,Material,Description,Article ID,Weight (kg),Quantity,Reserved
0,HDC,Material is Oiled,23048203,24469,52,NOT RESERVED
1,S235JR,Material is Oiled,23040547,16984,41,NOT RESERVED
2,S235JR,Material is Painted,23046057,9162,28,NOT RESERVED
3,DX51D +AZ150,Material is Oiled,23041966,12119,66,VANILLA
4,HDC,Material is Painted,23043884,17260,26,NOT RESERVED


### Parsing and Cleaning Numeric Ranges

Here, I wrote a small helper function to clean and convert all the messy numeric values from the reference dataset into usable numbers. It handles cases like numbers with commas or dots, single values, and ranges like 2.5-3.0. It also deals with inequalities like ≤5 or ≥10, returning them as (min, max) tuples. Basically, it makes sure that anything I have in those numeric columns can be used for calculations later without breaking anything. 

In [28]:
import re

def to_float(x):
    # Convert messy numbers (commas, etc.) into float.
    if pd.isna(x): return np.nan
    s = str(x).strip().replace(' ', '')
    # Handle 1.234,56 style numbers
    if '.' in s and ',' in s:
        s = s.replace('.', '').replace(',', '.')
    else:
        s = s.replace(',', '.')
    s = re.sub(r'[^0-9\.\-]', '', s)  # keep only digits, dot, minus
    try:
        return float(s)
    except:
        return np.nan

def parse_range(s):
    # Convert values like '2.5-3.0' or '≤5' into (min,max).
    if pd.isna(s): return (np.nan, np.nan)
    s = str(s).strip()
    if '-' in s or '–' in s:
        parts = re.split(r'[-–]', s)
        a = to_float(parts[0])
        b = to_float(parts[1]) if len(parts)>1 else a
        return (min(a,b), max(a,b))
    if '≤' in s or '<=' in s:
        return (np.nan, to_float(s))
    if '≥' in s or '>=' in s:
        return (to_float(s), np.nan)
    # fallback: single value
    v = to_float(s)
    return (v, v)

### Define a function to normalize each supplier file

Here I map their specific column names into a common schema.  
- Columns like `Thickness (mm)` and `Width (mm)` are normalized into min/max ranges.  
- `Quality/Choice`, `Description`, and other relevant business fields are preserved.  
- If a column does not exist for a supplier, it is filled with `NaN`.  

In [29]:
def process_supplier_df(df, source_name):
    out = pd.DataFrame()
    
    #Article ID
    if 'Article ID' in df.columns:
        out['article_id'] = df['Article ID']
    else:
        out['article_id'] = [f"{source_name}_{i+1}" for i in range(len(df))]
    
     #Grade
    if 'Grade' in df.columns:
        out['grade'] = df['Grade'].astype(str).str.upper().str.strip()
    elif 'Material' in df.columns:
        out['grade'] = df['Material'].astype(str).str.upper().str.strip()
    else:
        out['grade'] = None

    #Quality
    if 'Quality/Choice' in df.columns:
        out['quality_choice'] = df['Quality/Choice'].apply(lambda x: str(x).strip() if pd.notna(x) else np.nan)
    else:
        out['quality_choice'] = np.nan

    #Description
    if 'Description' in df.columns:
        out['description'] = df['Description'].apply(lambda x: str(x).strip() if pd.notna(x) else np.nan)
    else:
        out['description'] = np.nan
    
    #Finish
    if 'Finish' in df.columns:
        out['finish'] = df['Finish'].astype(str).str.strip()
    else:
        out['finish'] = None
    
    #Thickness
    if 'Thickness (mm)' in df.columns:
        out[['thickness_min','thickness_max']] = df['Thickness (mm)'].apply(lambda x: pd.Series(parse_range(x)))
    else:
        out[['thickness_min','thickness_max']] = (np.nan, np.nan)
    
    #Width
    if 'Width (mm)' in df.columns:
        out[['width_min','width_max']] = df['Width (mm)'].apply(lambda x: pd.Series(parse_range(x)))
    else:
        out[['width_min','width_max']] = (np.nan, np.nan)
    
    #Quantity
    if 'Quantity' in df.columns:
        out['quantity'] = df['Quantity'].apply(to_float)
    else:
        out['quantity'] = None
    
    #Weight
    if 'Gross weight (kg)' in df.columns:
        out['weight_kg'] = df['Gross weight (kg)'].apply(to_float)
    elif 'Weight (kg)' in df.columns:
        out['weight_kg'] = df['Weight (kg)'].apply(to_float)
    else:
        out['weight_kg'] = None

    if 'Reserved' in df.columns:
        out['reserved'] = df['Reserved'].astype(str).str.strip()
    else:
        out['reserved'] = None
    
    #Mechanical properties (supplier 1 only)
    if 'RP02' in df.columns:
        out['rp02'] = df['RP02'].apply(to_float)
    else:
        out['rp02'] = None
    
    if 'RM' in df.columns:
        out['rm'] = df['RM'].apply(to_float)
    else:
        out['rm'] = None
    
    if 'AG' in df.columns:
        out['ag'] = df['AG'].apply(to_float)
    else:
        out['ag'] = None
    
    if 'AI' in df.columns:
        out['ai'] = df['AI'].apply(to_float)
    else:
        out['ai'] = None

    out['source'] = source_name
    
    return out

### Joining into a single table called inventory_dataset.

In [33]:
df1 = process_supplier_df(s1, "supplier1")
df2 = process_supplier_df(s2, "supplier2")

#Defining the final schema 
final_cols = [
    "article_id","grade","finish","quality_choice","description",
    "thickness_min","thickness_max","width_min","width_max",
    "quantity","weight_kg","reserved","rp02","rm","ag","ai","source"
]

#Reindex both DataFrames so they have the same columns
df1 = df1.reindex(columns=final_cols)
df2 = df2.reindex(columns=final_cols)

#Concatenate 
inventory = pd.concat([df1, df2], ignore_index=True)

C:\Users\dd\AppData\Local\Temp\ipykernel_5456\4206145.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  inventory = pd.concat([df1, df2], ignore_index=True)


### Save the cleaned dataset

I save with `utf-8-sig` so that German characters (e.g., *geglüht*, *Längs*) appear correctly in Excel.

In [45]:
import os

# Define the path to the existing output folder at project root
output_folder = "../outputs"  # '..' goes one level up from notebook folder

# Make sure the folder exists 
os.makedirs(output_folder, exist_ok=True)

# Save CSV
inventory.to_csv(os.path.join(output_folder, "inventory_dataset.csv"), index=False, encoding="utf-8-sig")
print("File saved in the outputs folder")

File saved in the outputs folder


### Validation and profiling

Finally, I check data consistency and completeness.

In [37]:
print("Rows per supplier:")
print(inventory['source'].value_counts())

print("\nMissing values per column:")
print(inventory.isna().sum())

print("\nThickness stats:")
print(inventory[['thickness_min','thickness_max']].describe())

print("\nWidth stats:")
print(inventory[['width_min','width_max']].describe())

print("\nPreview of final dataset:")
display(inventory.head(10))

Rows per supplier:
source
supplier1    50
supplier2    50
Name: count, dtype: int64

Missing values per column:
article_id         0
grade              0
finish            50
quality_choice    50
description        0
thickness_min     50
thickness_max     50
width_min         50
width_max         50
quantity           0
weight_kg          0
reserved          50
rp02              50
rm                50
ag                50
ai                50
source             0
dtype: int64

Thickness stats:
       thickness_min  thickness_max
count      50.000000      50.000000
mean        2.508800       2.508800
std         0.273112       0.273112
min         2.010000       2.010000
25%         2.292500       2.292500
50%         2.510000       2.510000
75%         2.750000       2.750000
max         2.980000       2.980000

Width stats:
         width_min    width_max
count    50.000000    50.000000
mean   1075.000000  1075.000000
std      48.968961    48.968961
min    1000.000000  1000.000000
25

,article_id,grade,finish,quality_choice,description,thickness_min,thickness_max,width_min,width_max,quantity,weight_kg,reserved,rp02,rm,ag,ai,source
0,supplier1_1,C200S,gebeizt und geglüht,3rd,Längs- oder Querisse,2.77,2.77,1100.0,1100.0,0.00,13983.0,None,333.6,606.2,16.11,0.0054,supplier1
1,supplier1_2,C300S,ungebeizt,3rd,Längs- oder Querisse,2.65,2.65,1075.0,1075.0,0.00,13047.0,None,717.7,0.0,16.11,0.0046,supplier1
2,supplier1_3,C100S,gebeizt und geglüht,3rd,Kantenfehler - FS-Kantenrisse,2.20,2.20,1100.0,1100.0,10.84,14155.0,None,368.9,0.0,0.00,0.0061,supplier1
3,supplier1_4,C100S,gebeizt,2nd,Längs- oder Querisse,2.86,2.86,1100.0,1100.0,22.87,11381.0,None,368.9,601.7,0.00,0.0062,supplier1
4,supplier1_5,C300S,ungebeizt,1st,Sollmasse (Gewicht) unterschritten,2.88,2.88,1050.0,1050.0,22.87,10072.0,None,0.0,1213.0,0.00,0.0041,supplier1
5,supplier1_6,C200S,gebeizt,2nd,Längs- oder Querisse,2.51,2.51,1000.0,1000.0,0.00,12863.0,None,0.0,606.2,0.00,0.0059,supplier1
6,supplier1_7,C100S,gebeizt,1st,Sollmasse (Gewicht) unterschritten,2.69,2.69,1150.0,1150.0,10.84,8936.0,None,0.0,606.2,15.05,0.0066,supplier1
7,supplier1_8,C300S,gebeizt und geglüht,2nd,Sollmasse (Gewicht) unterschritten,2.98,2.98,1000.0,1000.0,22.87,9810.0,None,0.0,601.7,15.05,0.0042,supplier1
8,supplier1_9,C200S,gebeizt,3rd,Längs- oder Querisse,2.03,2.03,1050.0,1050.0,22.87,5491.0,None,368.9,601.7,0.00,0.0064,supplier1
9,supplier1_10,C300S,gebeizt,3rd,Längs- oder Querisse,2.54,2.54,1050.0,1050.0,0.00,11415.0,None,368.9,606.2,15.05,0.0067,supplier1


### Creating a ReadMe File

In [1]:
import os

# Create outputs folder if it doesn't exist
os.makedirs("outputs", exist_ok=True)

# The README content
readme_content = """
# README — Task A: Cleaning and Combining Supplier Data

## What I Did
The goal of this task was to clean and combine two supplier datasets into one final inventory dataset named inventory_dataset.csv.

Supplier 1 and Supplier 2 provide similar information, but in different formats and with slightly different fields. Supplier 1 includes details like quality, grade, finish, thickness, width, description, weight, quantity, and some mechanical properties. Supplier 2 provides material, description, weight, quantity, and reserved status.

I cleaned both datasets, standardized their columns, handled missing values, and fixed encoding issues so that the data is consistent, complete, and ready for further analysis.

## Steps I Followed
I first read both datasets and checked that the files loaded correctly. I then created a function to process each supplier dataset individually. This function maps the supplier-specific column names to a common format, ensures consistent naming using snake_case, and fills missing columns with empty values when needed.

For columns that exist in one supplier but not in the other, I kept them as empty so that both datasets align when combined. I also preserved important free-text columns like description. After cleaning, I combined the two datasets into a single inventory dataset.

I paid special attention to German characters such as ü, ö, and ß, which sometimes display incorrectly due to encoding issues. By saving the final CSV in UTF-8 format with a BOM (utf-8-sig), these characters now display correctly in Excel.

Finally, I normalized the reserved column to True/False to make the dataset more business-friendly. I also validated the dataset to check for missing values, row counts per supplier, and basic statistics for numeric columns.

Issues Faced and How I Solved Them
German characters like ü were appearing incorrectly as Ã¼. This was caused by encoding problems when saving or opening the file in Excel. I resolved this by saving the final CSV with UTF-8 encoding including a BOM, which ensures that Excel correctly displays German letters.

The description column appeared empty in some cases. This was because sometimes Excel files contain cells that look filled but are read as blank by pandas. I made sure to preserve any existing text and only leave cells empty when they are genuinely missing.

The Quality/Choice column from Supplier 1 was showing NaN in Jupyter Notebook. This was caused by differences in column naming or extra spaces in the original dataset. I mapped it carefully to the final column name quality_choice and ensured whitespace was removed so that the values display correctly. For Supplier 2, this column remains empty because it is not provided, which is expected.

## Assumptions I Made
Since Supplier 2 does not provide thickness, width, finish, or mechanical properties, I left these fields empty for their rows. Similarly, Supplier 1 does not provide Article ID, so I left it blank for Supplier 1 rows.

I assumed that Description, even if it is free-text or in German, is useful information, so I preserved it as-is for both suppliers.

I standardized all column names to snake_case for consistency and clarity. For missing values in any column, I kept them empty rather than trying to guess or fill values, to maintain data accuracy.

For the Reserved column in Supplier 2, I normalized the values to True/False to make it easier to interpret and analyze later.

Finally, I assumed that the final dataset should retain all available information from both suppliers without losing any key columns, even if some are empty for one supplier.

## Outcome
The final inventory dataset has all columns aligned, cleaned, and standardized. Missing values are handled properly. German characters display correctly in Excel. Free-text columns like description are preserved, and numeric fields have been validated. The dataset is ready for further analysis or use in Task B.

This work demonstrates careful data cleaning, thoughtful handling of missing values and special characters, and attention to business relevance, making the dataset reliable and useful for analysis.

# Setup and Run Instructions
1. Python Environment

These notebooks are written in Python 3.

It is recommended to use Conda to manage your environment and dependencies.

## To create a new environment, you can run:

conda create -n vanillasteel python=3.10
conda activate vanillasteel


## Install required packages using pip or conda. Here are the main libraries used in the notebooks:

pandas

numpy

openpyxl

matplotlib

seaborn

scikit-learn

re (regular expressions)

## Any other standard Python libraries

You can install them with:

pip install pandas numpy openpyxl matplotlib seaborn scikit-learn

## 2. Folder Structure

Make sure your project folder is organized like this:

vanillasteel-assessment/
│
├── notebooks/          # All Task A, Task B, Bonus Task notebooks
├── data/               # Supplier files and any other input data
├── outputs/            # Folder to save processed CSV files
├── README-taskA.txt
├── README-taskB.txt
├── bonus_task_output/  # Folder to save outputs from the bonus task
├── README-Bonus.md


The data folder contains supplier files (e.g., supplier_data1.xlsx, supplier_data2.xlsx) needed for Task A.

The outputs folder is where the CSV files generated by the notebooks (e.g., inventory_dataset.csv, top3.csv) will be saved. Ensure this folder already exists.

## 3. Running the Notebooks
## Task A

Open the Task A notebook in Jupyter Notebook.

Run each cell sequentially from top to bottom.

The final output, inventory_dataset.csv, will be saved in the outputs folder.

Note: CSV files are saved using UTF-8 with BOM (utf-8-sig) encoding so that special characters like German letters (ü, ö, ß) display correctly in Excel.
## 4. Notes on CSV and Excel

If opening CSV files in Excel, make sure to choose UTF-8 encoding to view German characters correctly.

In Excel:

Open Excel.

Go to Data → Get Data → From Text/CSV.

Select the file and set File Origin to UTF-8.

This ensures columns like description and finish display correctly without garbled text.

## 5. Optional

If using Jupyter Notebook, make sure your kernel is set to the Python environment you installed with the required libraries.

Always check the outputs folder to ensure CSV files are generated correctly after running the notebooks.
"""

# Save README.txt in outputs folder
output_folder = "../outputs"
readme_path = os.path.join(output_folder, "README-Task A.md")
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_content)
print(f"README.md successfully saved at: {readme_path}")

README.md successfully saved at: ../outputs\README-Task A.md
